In [3]:
import numpy as np
import os
import time

In [4]:
#reads fname, returns a)list of params, b)status: cars,runs,reward
#note that runs are enriched with a final flag (run_to_assign)
def read_input(fname):
    raw_data = open(fname,'r').read().split('\n')
    [nrow, ncol, ncar, nride, ontime_bonus, max_time] = [int(x) for x in raw_data[0].split(' ')]
    cars = [[i, 0, 0, 0] for i in range(ncar)]
    runs = [[int(x) for x in line.split(' ')] + [True] for line in raw_data[1:-1]]
    return [nrow, ncol, ncar, nride, ontime_bonus, max_time], [cars, runs, 0]

In [9]:
#manhattan
def distance(start, stop):
    return abs(start[0] - stop[0]) + abs(start[1] - stop[1])

#returns ranking, arrival time, local_reward
#ranking: heuristic to select a run-car pair to assign
def compute_cost(run, car, ontime_bonus, max_time):
    distance_to_start = distance(run[:2], car[1:3])
    waiting_time = max(0, run[4] - (car[3] + distance_to_start))
    bonus = ontime_bonus if (run[4] - (car[3] + distance_to_start)) >= 0 else 0
    final_time = car[3] + distance_to_start + waiting_time + distance(run[:2], run[2:4])

    if final_time > max_time or final_time >= run[5]:
        #this car-run pair can't be allocated, returning default low ranking
        return -max_time, -1, 0
    else:
        return -distance_to_start - waiting_time + bonus + distance(run[:2], run[2:4]), final_time, bonus + distance(run[:2], run[2:4])

#create the initial raking matrix - which will be locally updated after each assignment
def init(reward, runs, cars, ontime_bonus, max_time):
    ranks = np.zeros([len(runs), len(cars)])

    for ir, r in enumerate(runs):
        for ic, c in enumerate(cars):
            ranks[ir][ic] = compute_cost(r, c, ontime_bonus, max_time)[0]

    return ranks

#update the rankings, greedily pick the best pairing, update state
def smart_step(ranks, reward, runs, cars, ontime_bonus, max_time, last_car_id, last_run_id):
    if(last_car_id != -1):

        #rule out the already allocated run
        for ic, c in enumerate(cars):
            ranks[last_run_id][ic] = -max_time

        #update the ranking column for the car that was just 'used'
        for ir, r in enumerate(runs):
            ranks[ir][last_car_id] = compute_cost(r, cars[last_car_id], ontime_bonus, max_time)[0] if runs[ir][6] else -max_time

    brun, bcar = divmod(np.argmax(ranks), len(cars))

    if ranks[brun][bcar] == -max_time:
        #out of allocable runs
        return [None, None, None, None, -1, -1]

    #now I have the best greedy option, mark the run as allocated
    runs[brun][6] = False
    arrival_time, local_reward = compute_cost(runs[brun], cars[bcar], ontime_bonus, max_time)[1:]
    cars[bcar] = np.array([cars[bcar][0], runs[brun][2], runs[brun][3], arrival_time])
    reward += local_reward 
    
    return ranks, reward, runs, cars, bcar, brun

In [10]:
def solve_problem(fname):
    tick = time.time()
    print(fname, "started at", time.strftime("%H:%M:%S", time.gmtime(tick)))

    [nrow, ncol, ncar, nride, ontime_bonus, max_time], [cars, runs, reward] = read_input(fname)

    ranks = init(reward, runs, cars, ontime_bonus, max_time)
    best_car, best_run, prev_rew = -1, -1, -1
    results = {}

    while(True):
        ranks, reward, runs, cars, best_car, best_run = smart_step(ranks, reward, runs, cars, ontime_bonus, max_time, best_car, best_run)

        if best_run == -1:
            break

        prev_rew = reward
        results.setdefault(best_car, []).append(best_run)

    fname_out = open(fname[:-3] + '.out', 'w')

    for k, v in results.items():
        fname_out.write(str(len(v)) + ' ' + ' '.join([str(i) for i in v]) + '\n')
        
    for i in range(len(results.keys()), ncar):
        fname_out.write('0\n')
        
    print('solved', fname, 'with reward of', prev_rew, 'and time', time.strftime("%H:%M:%S", time.gmtime(time.time() - tick)))

In [11]:
def check_solutions(fname):
    [nrow, ncol, ncar, nride, ontime_bonus, max_time], [cars, runs, reward] = read_input(fname)

    out_fname = fname.replace('.in', '.out')
    raw_data = open(out_fname,'r').read().split('\n')
    
    results = [[int(ride) for ride in car.split(' ')[1:]] for car in raw_data[:-1]]

    test_total_reward = 0
    for k, v in enumerate(results):
        test_pos = [0, 0]
        test_reward = 0
        test_time = 0
        for r in v:
            #print("Car does the ride", r)
            test_time += distance(test_pos, runs[r][0:2])
            #print("Car moves to ride at time {} ### ride starts at {} and should finish at {}".format(test_time, runs[r][4], runs[r][5]))
            if test_time < runs[r][4]:
                test_reward += ontime_bonus
                #print("Bonus! Car waits till", runs[r][4])
            test_time = max(runs[r][4], test_time)
            ride_distance = distance(runs[r][0:2], runs[r][2:4]) 
            test_reward += ride_distance
            test_time += ride_distance
            test_pos = runs[r][2:4]
            #print("Car does the ride to time {} ### ride reward {}, total reward {}".format(test_time, ride_distance, test_reward))
            assert(test_time <= runs[r][5])
        test_total_reward += test_reward
        #print("{} Car reward {}, partial reward {}".format(k, test_reward, test_total_reward))
    print(test_total_reward)


In [12]:
#Just for quick notebook-testing
def get_inputs():
    folder = './data/'

    return [folder + x for x in sorted(os.listdir(folder)) if x[-3:] == '.in']

for fname in get_inputs()[0:2]:
    solve_problem(fname)
    check_solutions(fname)

./data/a_example.in started at 22:20:28
(3, 2) [[ 4.  4.]
 [-1. -1.]
 [ 0.  0.]]
(3, 2) [[-10. -10.]
 [-10.  -1.]
 [-10.   0.]]
(3, 2) [[-10. -10.]
 [-10.   1.]
 [-10. -10.]]
(3, 2) [[-10. -10.]
 [-10. -10.]
 [-10. -10.]]
solved ./data/a_example.in with reward of 10 and time 00:00:00
10
./data/b_should_be_easy.in started at 22:20:28
(300, 100) [[   573.    573.    573. ...    573.    573.    573.]
 [-21094. -21094. -21094. ... -21094. -21094. -21094.]
 [-23946. -23946. -23946. ... -23946. -23946. -23946.]
 ...
 [-19611. -19611. -19611. ... -19611. -19611. -19611.]
 [-12678. -12678. -12678. ... -12678. -12678. -12678.]
 [ -3365.  -3365.  -3365. ...  -3365.  -3365.  -3365.]]
(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-19645. -21094. -21094. ... -21094. -21094. -21094.]
 [-22497. -23946. -23946. ... -23946. -23946. -23946.]
 ...
 [-18162. -19611. -19611. ... -19611. -19611. -19611.]
 [-11229. -12678. -12678. ... -12678. -12678. -12678.]
 [ -1916.  -3365.  -3365. .

(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -21094. -21094. -21094.]
 [-25000. -25000. -25000. ... -23946. -23946. -23946.]
 ...
 [-25000. -25000. -25000. ... -19611. -19611. -19611.]
 [-25000. -25000. -25000. ... -12678. -12678. -12678.]
 [-25000. -25000. -25000. ...  -3365.  -3365.  -3365.]]
(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -21094. -21094. -21094.]
 [-25000. -25000. -25000. ... -23946. -23946. -23946.]
 ...
 [-25000. -25000. -25000. ... -19611. -19611. -19611.]
 [-25000. -25000. -25000. ... -12678. -12678. -12678.]
 [-25000. -25000. -25000. ...  -3365.  -3365.  -3365.]]
(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -21094. -21094. -21094.]
 [-25000. -25000. -25000. ... -23946. -23946. -23946.]
 ...
 [-25000. -25000. -25000. ... -19611. -19611. -19611.]
 [-25000. -25000. -25000. ... -12678. -12678. -12678.]
 [-25000. -2500

(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -21094. -21094. -21094.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 ...
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -12678. -12678. -12678.]
 [-25000. -25000. -25000. ...  -3365.  -3365.  -3365.]]
(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -21094. -21094. -21094.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 ...
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -12678. -12678. -12678.]
 [-25000. -25000. -25000. ...  -3365.  -3365.  -3365.]]
(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -21094. -21094. -21094.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 ...
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -12678. -12678. -12678.]
 [-25000. -2500

(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 ...
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]]
(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 ...
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]]
(300, 100) [[-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 ...
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -25000. -25000. ... -25000. -25000. -25000.]
 [-25000. -2500